In [1]:
#Install all the Reqiuired Libraries and Packages 
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')

In [2]:
#All the Required Packages and Libraies are installed.
import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
#from keras.utils import to_categorical 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [3]:
os.listdir(path=r'C:\Users\Pooja\Downloads\Webapp Project_3\data')
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = r'C:\Users\Pooja\Downloads\Webapp Project_3\data'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

372

In [4]:
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [5]:
#Next Step is In-Depth Visualisation of Audio Fiels and its certain features to plot for.
#They are the Plotting Functions to be called later. 
def plot_signals(signals):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Time Series' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def plot_fft(fft):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Fourier Transform' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            data = list(fft.values())[i]
            Y,freq = data[0] , data[1]
            axes[x,y].set_title(list(fft.keys())[i])
            axes[x,y].plot(freq , Y)
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1
    
def plot_fbank(fbank):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Filter Bank Coefficients' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(fbank.keys())[i])
            axes[x,y].imshow(list(fbank.values())[i],cmap='hot', interpolation = 'nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1
            
def plot_mfccs(mfccs):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Mel Frequency Capstrum  Coefficients' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(mfccs.keys())[i])
            axes[x,y].imshow(list(mfccs.values())[i],
                             cmap='hot', interpolation = 'nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def calc_fft(y,rate):
    n = len(y)
    freq = np.fft.rfftfreq(n , d= 1/rate)
    Y= abs(np.fft.rfft(y)/n)
    return(Y,freq)

In [6]:
#Now Cleaning Step is Performed where:
#DOWN SAMPLING OF AUDIO FILES IS DONE  AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
#MASK IS TO REMOVE UNNECESSARY EMPTY VOIVES AROUND THE MAIN AUDIO VOICE 
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [7]:
#The clean Audio Files are redirected to Clean Audio Folder Directory 
import glob,pickle
for file in tqdm(glob.glob(r'data\\**\\*.wav')):
    file_name = os.path.basename(file)
    signal , rate = librosa.load(file, sr=16000)
    mask = envelope(signal,rate, 0.0005)
    wavfile.write(filename= 'clean_speech\\'+str(file_name), rate=rate,data=signal[mask])

100%|████████████████████████████████████████████████████████████████████████████████| 372/372 [00:39<00:00,  9.53it/s]


In [8]:
 #Feature Extraction of Audio Files Function 
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

In [9]:
birds = {'11713':'Dendrocopos major',
         '11846':'Chloris chloris',
         '12577':'Corvus frugilegus',
         '12578':'Coccothraustes coccothraustes',
         '12876':'Columba palumbus',
         '12996':'Delichon urbicum',
         '13164':'Apus apus',
         '13602':'Sitta europaea',
         '13608':'Corvus monedula',
         '13609':'Phoenicurus ochruros',
         '13610':'Turdus merula',
         '14172':'Turdus pilaris',
         '14212':'Passer montanus',
         '14213':'Phylloscopus trochilus',
         '14231':'Phylloscopus collybita',
         '14426':'Phoenicurus phoenicurus',
         '14442':'Motacilla alba',
         '14518':'Erithacus rubecula',
         '14844':'Streptopelia decaocto',
         '15245':'Parus major',
         '15269':'Parus caeruleus',
         '15270':'Alauda arvensis',
         '18125':'Luscinia luscinia',
         '18247':'Garrulus glandarius',
         '18344':'Turdus philomelos',
         '18387':'Pica pica',
         '18388':'Troglodytes troglodytes',
         '18483':'Carduelis carduelis',
         '18484':'Sturnus vulgaris',
         '20420':'Emberiza citrinella'}

observed_birds = ['Dendrocopos major','Chloris chloris','Corvus frugilegus','Coccothraustes coccothraustes'
,'Columba palumbus','Delichon urbicum','Apus apus','Sitta europaea','Corvus monedula','Phoenicurus ochruros',
'Turdus merula','Turdus pilaris','Passer montanus','Phylloscopus trochilus','Phylloscopus collybita','Phoenicurus phoenicurus',
'Motacilla alba','Erithacus rubecula','Streptopelia decaocto','Parus major','Parus caeruleus','Alauda arvensis',
'Luscinia luscinia','Garrulus glandarius','Turdus philomelos','Pica pica','Troglodytes troglodytes','Carduelis carduelis',
  'Sturnus vulgaris','Emberiza citrinella']


In [10]:
#Load the data and extract features for each sound file
from glob import glob
import os
import glob
def load_data(test_size=0.33):
    x,y=[],[]
    answer = 0
    for file in glob.glob(r'C:\Users\Pooja\Downloads\Webapp Project_3\clean_speech/*.wav'):
        file_name=os.path.basename(file)
        bird=birds[file_name.split("-")[0]]
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append([bird,file_name])
          
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [11]:
#Split the dataset
import librosa
import numpy as np
x_train,x_test,y_trai,y_tes=load_data(test_size=0.33)
print(np.shape(x_train),np.shape(x_test), np.shape(y_trai),np.shape(y_tes))
y_test_map = np.array(y_tes).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_trai).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

C:\Users\Pooja\miniconda3\envs\DL04\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Pass y=[ 0.00320275  0.00493622  0.00068311 ...  0.00337798 -0.0020317
 -0.02397621] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\Pooja\miniconda3\envs\DL04\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Pass y=[ 0.09613023 -0.3961642  -0.4828292  ... -0.05293456  0.07684665
 -0.03233762] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\Pooja\miniconda3\envs\DL04\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Pass y=[ 0.01178192  0.00665098  0.00942267 ...  0.15028012 -0.03269928
 -0.18527712] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\Pooja\miniconda3\envs\DL04\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Pass y=[ 0.01282268  0.01326143  0.00113445 ...  0.00465156 -0.0103512
 

(249, 180) (123, 180) (249, 2) (123, 2)
(249,) (123,)
15270-6.wav
18483-0.wav
18484-3.wav
12996-11.wav
13610-15.wav
14442-8.wav
14213-7.wav
18125-7.wav
14518-11.wav
13164-0.wav
13164-6.wav
14442-9.wav
18125-2.wav
14172-2.wav
14212-6.wav
18388-1.wav
18344-2.wav
14213-3.wav
15269-9.wav
15270-7.wav
13609-9.wav
18387-2.wav
14213-2.wav
14518-0.wav
11846-0.wav
13608-6.wav
11846-6.wav
12577-3.wav
14212-5.wav
12876-8.wav
18344-11.wav
13608-9.wav
20420-3.wav
11846-4.wav
15245-9.wav
13608-2.wav
13608-8.wav
13609-10.wav
20420-7.wav
18247-10.wav
12577-9.wav
15269-10.wav
11846-13.wav
11713-10.wav
12578-2.wav
14426-10.wav
12578-8.wav
13608-4.wav
11713-15.wav
13602-14.wav
14231-8.wav
13164-10.wav
14212-13.wav
12876-15.wav
13610-0.wav
18484-6.wav
18483-13.wav
14172-9.wav
13602-13.wav
18387-5.wav
13608-0.wav
18388-3.wav
14213-15.wav
12996-9.wav
14426-5.wav
18484-7.wav
12996-7.wav
18344-1.wav
14442-2.wav
11846-3.wav
11713-5.wav
12876-2.wav
18344-4.wav
14844-5.wav
13602-12.wav
12996-1.wav
14844-10.wav
13

C:\Users\Pooja\miniconda3\envs\DL04\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Pass y=[-0.25197053 -0.13521709  0.26705328 ... -0.00633611 -0.00680339
 -0.00694084] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\Pooja\miniconda3\envs\DL04\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Pass y=[-0.0016975   0.00063337  0.00244962 ...  0.00017177 -0.00050704
 -0.00173343] as keyword args. From version 0.10 passing these as positional arguments will result in an error


In [12]:
#Get the shape of the training and testing datasets
# print((x_train.shape[0], x_test.shape[0]))
print((x_train[0], x_test[0]))
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(array([-3.75216583e+02,  9.08760452e+00, -4.05933113e+01,  4.82057648e+01,
       -4.29607821e+00, -2.56625509e+00,  1.83580341e+01,  1.40731859e+01,
       -2.50447363e-01,  1.26564608e+01,  4.12815094e+00,  1.98092866e+00,
        1.38946753e+01,  8.64766407e+00,  4.61264992e+00,  4.86101627e+00,
        2.87312341e+00,  1.32341785e+01,  5.41235864e-01,  4.67423058e+00,
        6.99411869e+00,  3.02490473e+00,  3.81490541e+00,  5.09836721e+00,
        1.04973388e+00,  3.55883813e+00,  1.26024187e+00,  5.45939589e+00,
        2.05124140e-01,  3.33283901e+00,  4.82140112e+00,  1.75135136e+00,
        6.22917473e-01,  4.11286592e+00,  8.32794487e-01,  1.15797436e+00,
        2.35452533e+00, -1.02584314e+00, -4.09764379e-01,  1.61714995e+00,
        5.22211850e-01,  6.46062732e-01,  6.75567448e-01,  6.77433014e-01,
        6.13461494e-01,  6.18360817e-01,  6.83500350e-01,  5.85923493e-01,
        6.06934190e-01,  5.49003541e-01,  4.89795208e-01,  4.88753974e-01,
        5.14137208e-01, 

In [13]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [14]:
#Train the model
model.fit(x_train,y_train)

C:\Users\Pooja\miniconda3\envs\DL04\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:612: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  "Got `batch_size` less than 1 or larger than "


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [15]:
#SAVING THE MODEL
import pickle
# Save the Modle to file in the current working directory
#For any new testing data other than the data in dataset

Pkl_Filename = "Bird_Voice_Detection_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [16]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Emotion_Voice_Detection_Model = pickle.load(file)

Emotion_Voice_Detection_Model

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [17]:
x_test

array([[-2.59268188e+02, -5.51287880e+01, -5.31530457e+01, ...,
         5.76424832e-03,  5.80405875e-04,  2.95179398e-05],
       [-3.13150543e+02,  1.39984818e+01, -8.04030685e+01, ...,
         1.46586724e-04,  4.00798235e-05,  3.99388227e-06],
       [-3.80276215e+02, -1.99838009e+01, -6.28202362e+01, ...,
         3.38138576e-04,  1.12091053e-04,  1.45811964e-05],
       ...,
       [-1.83989410e+02, -3.72470856e+01, -1.25865524e+02, ...,
         3.37081701e-02,  1.37956394e-02,  5.64863323e-04],
       [-3.10141052e+02, -5.84712029e+01, -2.80029354e+01, ...,
         1.20338522e-01,  1.87553614e-02,  7.28610205e-04],
       [-4.21299408e+02, -7.52609482e+01,  2.51218185e+01, ...,
         8.71720552e-01,  1.01649165e-01,  4.05945210e-03]])

In [18]:
#predicting :
y_pred=Emotion_Voice_Detection_Model.predict(x_test)
y_pred

array(['Alauda arvensis', 'Carduelis carduelis', 'Sturnus vulgaris',
       'Delichon urbicum', 'Turdus merula', 'Motacilla alba',
       'Phylloscopus trochilus', 'Luscinia luscinia',
       'Erithacus rubecula', 'Apus apus', 'Apus apus', 'Motacilla alba',
       'Luscinia luscinia', 'Turdus pilaris', 'Passer montanus',
       'Troglodytes troglodytes', 'Turdus philomelos',
       'Phylloscopus trochilus', 'Parus caeruleus', 'Alauda arvensis',
       'Phoenicurus ochruros', 'Pica pica', 'Phylloscopus trochilus',
       'Erithacus rubecula', 'Chloris chloris', 'Corvus monedula',
       'Chloris chloris', 'Corvus frugilegus', 'Passer montanus',
       'Columba palumbus', 'Turdus philomelos', 'Corvus monedula',
       'Emberiza citrinella', 'Chloris chloris', 'Parus major',
       'Corvus monedula', 'Corvus monedula', 'Phoenicurus ochruros',
       'Emberiza citrinella', 'Garrulus glandarius', 'Corvus frugilegus',
       'Parus caeruleus', 'Chloris chloris', 'Dendrocopos major',
       '

In [19]:
#Store the Prediction probabilities into CSV file 
import numpy as np
import pandas as pd
y_pred1 = pd.DataFrame(y_pred, columns=['predictions'])
y_pred1['file_names'] = test_filename
print(y_pred1)
y_pred1.to_csv('predictionfinal.csv')

                 predictions    file_names
0            Alauda arvensis   15270-6.wav
1        Carduelis carduelis   18483-0.wav
2           Sturnus vulgaris   18484-3.wav
3           Delichon urbicum  12996-11.wav
4              Turdus merula  13610-15.wav
..                       ...           ...
118      Carduelis carduelis  18483-12.wav
119           Motacilla alba   14442-0.wav
120                Apus apus   13164-9.wav
121  Phoenicurus phoenicurus   14426-4.wav
122                Pica pica   18387-8.wav

[123 rows x 2 columns]
